### Example 14 - Selecting Market Making Targets

In this example, we'll filter for good market making targets according to the following criteria:

1. **Liquidity** - the extent to which assets can be bought or sold at stable prices \(typically reflected by volume\).
2. **Profitable Spread Ratio** - whether the spread ratio \(ask / bid\) is usually profitable for an asset.
3. **Return** - the return on the funds we use for market making.
4. **Balanced Volume** - we'll only consider assets with reasonable volume on both sides of the book.
5. **Competition** - we'll limit ourselves to assets without too much competition.

As discussed in the text, we'll restrict our analysis to one day of the week \(Saturday\) over several months of history.

Note that we've already developed all of the tools we need to perform our analysis from previous chapters. This example, therefore, largely consists of choosing thresholds for existing tools.

In [1]:
# Standard imports
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
# EveKit imports`
from evekit.reference import Client
from evekit.util import convert_raw_time

In [2]:
# As in previous examples, we'll consider opportunities in The Forge at the busiest station in Jita.
# We'll look for market making targets which are consistently strong over an approximately five
# month historical period.  It's likely such assets will continue to be strong market making options
# in the near future but, of course, historical performance does not guarantee future returns.
#
sde_client = Client.SDE.get()
region_query = "{values: ['The Forge']}"
station_query = "{values: ['Jita IV - Moon 4 - Caldari Navy Assembly Plant']}"
region_id = sde_client.Map.getRegions(regionName=region_query).result()[0][0]['regionID']
station_id = sde_client.Station.getStations(stationName=station_query).result()[0][0]['stationID']
date_range = pd.date_range(datetime.datetime(2017, 1, 7), datetime.datetime(2017, 5, 20))
print("Using region_id=%d, station_id=%d from %s to %s" % (region_id, station_id, str(date_range[0]), str(date_range[-1])))

Using region_id=10000002, station_id=60003760 from 2017-01-07 00:00:00 to 2017-05-20 00:00:00


In [3]:
# We'll filter for liquidity first, starting from all available types.
#
market_types = Client.SDE.load_complete(sde_client.Inventory.getTypes, marketGroupID="{start: 0, end: 1000000000}")
market_type_map = {}
for x in market_types:
    market_type_map[x['typeID']] = x
len(market_types)

11781

In [4]:
# As we only want to consider Saturdays, we'll construct a revised date range and load market
# history for just those days.
#
from evekit.marketdata import MarketHistory
sat_date_range = [x for x in date_range if x.weekday() == 5]
market_history = MarketHistory.get_data_frame(dates=sat_date_range, types=market_type_map.keys(), regions=[region_id], 
                                              config=dict(local_storage=".", tree=True, skip_missing=True, verbose=True))

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [5]:
# Include the liquidity filter framework from previous examples.
#
def liquid_types(history, liquidp, verbose=False):
    # Result is a map from region to set of liquid types for that region
    # Iterate through all types contained in the history object
    liquid_map = {}
    count = 0
    # Iterate through all regions and types
    for next_region in history.region_id.unique():
        liquid_set = set()
        by_region = history[history.region_id == next_region]
        for next_type in by_region.type_id.unique():
            by_type = by_region[by_region.type_id == next_type]
            if liquidp(next_region, next_type, by_type):
                liquid_set.add(next_type)
            count += 1
            if count % 1000 == 0 and verbose:
                print("Tested %d (region, type) pairs" % count)
        liquid_map[next_region] = liquid_set
    return liquid_map

In [6]:
# In this example, we'll filter for assets which trade almost every day in our historic range, 
# have a reasonable number of orders, and a price averaged volume over a certain threshold.
# These criteria suggest matching assets will trade consistently over our date range at a
# price averaged volume large enough to yield reasonable profit.
#
def liquidity_filter(min_days, min_count, min_volume):
    def liquidp(region_id, type_id, history):
        return len(history) >= min_days and \
               len(history[history.order_count < min_count]) == 0 and \
               len(history[history.volume * history.avg_price < min_volume]) == 0
    return liquidp

In [7]:
# We'll use the following values to parameterize the liquidity filter.
#
# Minimum number of orders per day
min_count = 250
# Minimum price averaged volume
min_volume = 100000000
# Each type we consider must trade every day
min_values = len(market_history.index.unique())

In [8]:
# Now compute liquid types.  The liquidity filter returns a map from region to the
# set of liquid types in that region.
#
liquid_type_map = liquid_types(market_history, liquidity_filter(min_values, min_count, min_volume))
len(liquid_type_map[region_id])

138

Next, we want to eliminate any types which do not have profitable spreads for the majority of each day of trading.  Recall from the text that a spread is profitable if:

${p_a \over p_b} > {{1 + b}\over{1 - t - b}}$

where:

* $p_a$ - is the best ask
* $p_b$ - is the best bid
* $b$ - is the broker fee rate for posting a limit order
* $t$ - is the sales tax rate for filling a buy order

Moreover, we want to eliminate any types which do not meet a given return target for the majority of each trading day.  Since a type which meets a certain return target must also be profitable, we can combine these two filters into a single test.  From the text \(and using the same definitions as above\), we know that return is given by the equation:

${{p_a}\over{p_a \times t + b \times (p_a + p_b) + p_b}} - 1$

Since return may be different for each order book snapshot in a given day, we'll compute the median of the snapshot returns and use that value as our measured return for the day.  We'll retain the assets for which this measured return exceeds the target return for at least half the days in our trading range.  This will give us a good selection of asset types which are often profitable.  We'll set an initial return target of 5%.

To compute return, we'll also need values for sales tax rate and broker fee rate.  We'll set these to 1% and 2.5%, respectively, which are typical values at NPC stations with max skills.

In [9]:
# We normally won't be able to load the entire date range unless the type set is very small.  So for this
# example, we iterate through the date set and build a DataFrame giving the daily median return for each 
# type.
#
from evekit.marketdata import OrderBook
#
broker_rate = 0.025
sales_tax_rate = 0.01
#
targets = liquid_type_map[region_id]
spread_data = []
for next_date in sat_date_range:
    order_book = OrderBook.get_data_frame(dates=[next_date], types=targets, regions=[region_id], 
                                          config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                      fill_gaps=True, verbose=True))
    # Compute best bid and ask for each snapshot and type
    by_loc = order_book[order_book.location_id == station_id]
    bids = by_loc[by_loc.buy == True]
    asks = by_loc[by_loc.buy == False]
    bids_grouped = bids.groupby([bids.index, bids.type_id]).price.max()
    asks_grouped = asks.groupby([asks.index, asks.type_id]).price.min()
    bids_us = bids_grouped.unstack()
    asks_us = asks_grouped.unstack()
    #
    # Now compute the median ask/bid ratio for each type
    for type_id in targets:
        joined = pd.concat([asks_us.xs(type_id, axis=1).rename("ask"), bids_us.xs(type_id, axis=1).rename("bid")], axis=1)
        returns = (joined.ask / (joined.ask * sales_tax_rate + broker_rate * (joined.ask + joined.bid) + joined.bid)) - 1
        median = returns.median()
        spread_data.append(dict(day=next_date, type_id=type_id, median_return=median))
#
# Finally, we convert the spread ratio data into a dataframe
spread_df = pd.DataFrame(spread_data, index=[x['day'] for x in spread_data])

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [10]:
# We can now filter down to the assets which have a daily median return which exceeds our target
# for every day in our date range.  Obviously, you can increase the number of assets which pass
# the filter by lowering the return target.
#
return_target = 0.05
high_return = spread_df[spread_df.median_return > return_target]
targets = high_return.type_id.groupby(high_return.type_id).count()
targets = list(targets[targets >= len(sat_date_range)/2].index)
len(targets)

28

Market making requires both buying and selling the same asset type several times a day.  Therefore, the best assets for market making will have reasonable volume on both sides of the book.  Our next filter will infer trades for each of our assets over the given date range, then compute buy and sell trade volume.  If both volumes exceed a reasonable threshold, then we know there is likely enough trade volume on both sides of the book to support market making.

Since we need to infer trades, we'll first set up our trade inference code as in previous examples.

In [11]:
# In preparation for trade inference, we need to commpute a volume threshold series for each type
# which will help us distinguish between large orders and cancels.  Note that our threshold computer
# requires a window of five days which means we're missing four weekends prior to our start date.
# To fix this, we'll reload market history here.
#
ext_date_range = [datetime.datetime(2016, 12, 10), datetime.datetime(2016, 12, 17),
                  datetime.datetime(2016, 12, 24), datetime.datetime(2016, 12, 31)] + sat_date_range
ext_market_history = MarketHistory.get_data_frame(dates=ext_date_range, types=targets, regions=[region_id], 
                                                  config=dict(local_storage=".", tree=True, skip_missing=True, verbose=True))

Retrieving 2016-12-10 00:00:00...done
Retrieving 2016-12-17 00:00:00...done
Retrieving 2016-12-24 00:00:00...done
Retrieving 2016-12-31 00:00:00...done
Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [12]:
# Now we can compute the thresholds we need for trade inferrence
#
volume_thresh_multiplier = 0.04
volume_thresh_map = {}
for next_type in ext_market_history.groupby(ext_market_history.type_id):
    group_id = next_type[0]
    group_df = next_type[1]
    volume_thresh_map[group_id] = group_df.volume.rolling(window=5, center=False).mean() * volume_thresh_multiplier

In [13]:
# For our trade side filter, it suffices to just capture the per side volume.  This allows for a simplified 
# trade inference function.
#
# Compute the buy and sell side trade volume for the given type on the given date
# using the given order book.  Large trades and cancels are distinguished using
# the given volume threshold map.
#
def compute_side_volume(type_id, dt, order_book, volume_threshold_map):
    buy_volume = 0
    sell_volume = 0
    by_type = order_book[order_book.type_id == type_id]
    vol_limit = volume_threshold_map[type_id][dt]
    #
    # Iterate over consecutive book snapshots looking for order book changes.
    #
    snap_list = list(by_type.groupby(by_type.index))
    snap_pairs = zip(snap_list, snap_list[1:])
    for current, next in snap_pairs:
        current_snap = current[1]
        current_time = current[0]
        next_snap = next[1]
        next_time = next[0]
        # Look for volume changes.  These are trades.
        merged = pd.merge(current_snap, next_snap, on="order_id")
        changed_orders = merged[merged.volume_x != merged.volume_y]
        for next_change in changed_orders.index:
            # Create the trade object
            next_line = changed_orders.ix[next_change]
            amount = next_line.volume_x - next_line.volume_y 
            if next_line.buy_x:
                buy_volume += amount
            else:
                sell_volume += amount
        # Look for removed orders.  These are either a fully filled order or a cancel.
        removed_orders = set(current_snap.order_id).difference(set(next_snap.order_id))
        current_order_list = list(current_snap.order_id)
        for order_id in removed_orders:
            order_count = current_order_list.count(order_id)
            next_line = current_snap[current_snap.order_id == order_id].ix[current_time]
            # If the volume of a removed order does not exceed the threshold, then it's a trade
            if order_count > 1:
                # Handle duplicate orders found in some data
                next_line = next_line.iloc[0]
            if next_line.volume <= vol_limit:
                if next_line.buy:
                    buy_volume += next_line.volume
                else:
                    sell_volume += next_line.volume
    # Return result
    return (buy_volume, sell_volume)

In [14]:
# We'll now iterate through our date range, computing the buy and sell volume for each type on
# each day.
#
side_volume_data = []
for next_date in sat_date_range:
    order_book = OrderBook.get_data_frame(dates=[next_date], types=targets, regions=[region_id], 
                                          config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                      fill_gaps=True, verbose=True))
    order_book = order_book[order_book.location_id == station_id]
    # Compute buy/sell volume for each type
    for type_id in targets:
        buy_volume, sell_volume = compute_side_volume(type_id, next_date, order_book, volume_thresh_map)
        side_volume_data.append(dict(day=next_date, type_id=type_id, buy_volume=buy_volume, sell_volume=sell_volume))
#
# Finally, we convert the side volume data into a dataframe
side_volume_df = pd.DataFrame(side_volume_data, index=[x['day'] for x in side_volume_data])

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [15]:
# Now that we've inferred buy and sell volume, we'll need to determine acceptable thresholds
# for considering an asset well balanced on a given day.  In an ideal world, buy and sell volume would
# each make up half the day's volume.  This rarely happens, so instead we'll expect at least one side
# to carry a reasonable amount of volume.  We'll arbitrarily choose 20%.  That is, if both buy and sell
# volume make up at least 20% of the total volume then we'll call the day balanced.  Note, also, that
# the actual volume ratio gives us a hint on how much volume we could expect to transact on a given
# day (hint: it's the smaller of the two volumes).
#
# The following code determines which asset types are balanced for each day in the target date range.
#
balanced_volume_threshold = 0.20
side_volume_df_copy = side_volume_df.copy()
side_volume_df_copy['total_volume'] = side_volume_df_copy.buy_volume + side_volume_df_copy.sell_volume
side_volume_df_copy['buy_ratio'] = side_volume_df_copy.buy_volume / side_volume_df_copy.total_volume
side_volume_df_copy['sell_ratio'] = side_volume_df_copy.sell_volume / side_volume_df_copy.total_volume
buy_exceeds_threshold = side_volume_df_copy[side_volume_df_copy.buy_ratio > balanced_volume_threshold]
both_exceed_threshold = buy_exceeds_threshold[buy_exceeds_threshold.sell_ratio > balanced_volume_threshold]
#
# We can now view for each asset type how many days meet our requirements
#
day_vol_counts = both_exceed_threshold.groupby(both_exceed_threshold.type_id).day.count()
print(day_vol_counts[day_vol_counts == len(sat_date_range)])
#
# The index of this result represents our new target set
#
targets = day_vol_counts[day_vol_counts == len(sat_date_range)].index
len(targets)


type_id
5973     20
8027     20
8089     20
8105     20
8433     20
8529     20
11105    20
16303    20
16527    20
28362    20
28363    20
Name: day, dtype: int64


11

In [16]:
# It follows that lowering the balanced volume threshold would admit more types for consideration.
#
# We're now down to our final filter which is to look at competition on these remaining types.  You may recall
# from the previous example that we detected competition by looking for order price changes.  We reasoned that
# the number of orders which change in a given time period is a reasonable upper bound for the number of
# active market participants.  We can use this information either to decide a given asset is too active, or
# to help formulate a strategy for dealing with a number of other competitors.
#
# To create our final filter, we need our order change counter from the previous example.
#
# The following function counts the number of orders which have changed price in a given
# interval.  The result is a Pandas DataFrame indexed by start snapshot time and containing 
# the columns:
#
# time - the time when one or more orders changed
# type_id - type ID which changed
# change_count - the number of orders which changed
#
def count_order_changes(order_book, type_list, sample_interval, verbose=False):
    samples = order_book.resample(sample_interval)
    total_samples = len(samples)
    changes = []
    if verbose:
        print("Checking %d samples for market participants" % total_samples, flush=True)    
    count = 0
    #
    for sample_group in samples:
        #
        # Each group is a pair (sample_time, sample_dataframe)
        sample_time = sample_group[0]
        sample = sample_group[1]
        if verbose:
            print("X", end='', flush=True)
            count += 1
            if count % 72 == 0:
                print()
        #
        # Iterate through each type in the type list
        for next_type in type_list:
            # Reduce this sample by type
            by_type = sample[sample.type_id == next_type]
            # Group by orders
            orders = by_type.groupby(['order_id'])
            # Count the unique prices for each order, flag those orders with more than
            # one price in the samnple interval.
            changed = orders['price'].nunique() > 1
            # Count how many orders changed price at least once in the sample interval.
            count = changed[changed == True].count()
            # Save the number of orders which changed prices
            changes.append(dict(time=sample_time, type_id=next_type, change_count=count))
    if verbose:
        print(flush=True)
    return pd.DataFrame(changes, index=[x['time'] for x in changes])

In [17]:
# Once again, we'll evaluate our remaining targets against the order book for each day in
# our target range.  On each day, we'll accumulate order changes for our target types.
# Order changes are accumulated over a sampling interval which should be set according
# to how frequently you're willing to update orders.  For this example, we'll set the sampling
# interval to 30 minutes.  In other words, we'll plan to refresh our orders every 30 minutes.
# This will give us 48 change count samples for each asset type on each day.
#
change_count_data = []
for next_date in sat_date_range:
    order_book = OrderBook.get_data_frame(dates=[next_date], types=targets, regions=[region_id], 
                                          config=dict(local_storage=".", tree=True, skip_missing=True, 
                                                      fill_gaps=True, verbose=True))
    order_book = order_book[order_book.location_id == station_id]
    # Compute and store change count for this day
    change_count_data.append(count_order_changes(order_book, targets, '30min'))

Retrieving 2017-01-07 00:00:00...done
Retrieving 2017-01-14 00:00:00...done
Retrieving 2017-01-21 00:00:00...done
Retrieving 2017-01-28 00:00:00...done
Retrieving 2017-02-04 00:00:00...done
Retrieving 2017-02-11 00:00:00...done
Retrieving 2017-02-18 00:00:00...done
Retrieving 2017-02-25 00:00:00...done
Retrieving 2017-03-04 00:00:00...done
Retrieving 2017-03-11 00:00:00...done
Retrieving 2017-03-18 00:00:00...done
Retrieving 2017-03-25 00:00:00...done
Retrieving 2017-04-01 00:00:00...done
Retrieving 2017-04-08 00:00:00...done
Retrieving 2017-04-15 00:00:00...done
Retrieving 2017-04-22 00:00:00...done
Retrieving 2017-04-29 00:00:00...done
Retrieving 2017-05-06 00:00:00...done
Retrieving 2017-05-13 00:00:00...done
Retrieving 2017-05-20 00:00:00...done


In [22]:
# We now have change counts for every 30 minutes on every day in the test data range.
# The choice of acceptable competition is highly subjective but let's apply a bit more 
# knowledge of our trading pattern to try to create more useful results.
#
# First, let's collect all change counts into a single dataframe
#
all_changes = change_count_data[0].append(change_count_data[1:])
#
# We're filtering for trading on a Saturday, so let's assume trading hours from 1200 UTC
# to 2400 UTC.  This is just after down time to midnight EVE time.  In reality, it's likely
# we'll trade past midnight UTC, but to perform that analysis we'd need two order books for
# each day.  So for now, we'll go with these hours.
#
# Let's eliminate all data outside of these hours.
#
constrained = all_changes[all_changes.index.hour >= 12]
#
# What's left can now be used to measure change count behavior.  But how should we do this?
# There are many possibilities:
#
# 1. Average change count
# 2. Median change count
# 3. Max change count
# 4. Some other quantile of change count
#
# Average or median change count will give us some expectation of what a typical time period
# might look like in terms of competition, but we risk the danger of undershooting the number
# of orders we need to maintain if the average or mean is unusually low.  Conversely, a
# measure of max count shows how bad things could get.  If we wanted to be extremely conservative,
# we could use max as a guideline for competition.  Finally, we could use a quantile, say the 95%
# change count quantile.  This would tell us, for example, the max change count for 95% of the
# sample intervals.  Quantiles make it easy to identify maximums which are true outliers.
#
# Let's look at all of these measures before we make a decision.
#
pd.DataFrame({ 'Average': constrained.groupby(constrained.type_id).change_count.mean(),
               'Median' : constrained.groupby(constrained.type_id).change_count.median(),
               'Max': constrained.groupby(constrained.type_id).change_count.max(),
               '95%': constrained.groupby(constrained.type_id).change_count.quantile(0.95)})

,95%,Average,Max,Median
type_id,,,,
5973,11.0,4.245833,17,3
8027,9.0,2.975000,14,2
8089,6.0,1.633333,10,1
8105,7.0,2.475000,19,2
8433,5.0,1.491667,10,1
8529,4.0,1.452083,9,1
11105,6.0,1.797917,11,1
16303,8.0,2.797917,16,2
16527,6.0,1.981250,14,1


The average and median numbers look very promising while the max numbers show where we might see extreme competition.  The 95% measure shows a diversity that more or less reflects the max numbers, but is about 60% smaller.  This suggests the maximums are likely infrequent outliers.

At this point, we're left with a very subjective decision.  For the sake of completing this example, let's choose to accept any type in which there are fewer than 10 changes in 30 minutes at the 95% interval.  This means that we know have our final set of types on which we'll attempt to make a market.

Further analysis is possible using the techniques described in the previous example.  For example, we could graph change data to see what times of day are most active for these types.  We'll leave further analsysis to the reader.

Eliminating 95% changes at 10 or higher and mapping to type names gives us our final list:

In [24]:
less_10_count = constrained.groupby(constrained.type_id).change_count.quantile(0.95) < 10
for x in less_10_count[less_10_count == True].index:
    print(market_type_map[x]['typeName'])

Prototype 'Arbalest' Rapid Light Missile Launcher
Arbalest Compact Light Missile Launcher
'Arbalest' Heavy Missile Launcher
Medium Azeotropic Restrained Shield Extender
Large F-S9 Regolith Compact Shield Extender
Vortex Compact Magnetic Field Stabilizer
'Halcyon' Core Equalizer I
'Arbalest' Rocket Launcher I
